In [ ]:
pip install music21

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install pandas 

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 19 kB/s eta 0:00:01
     |████████████████████████████████| 346 kB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 507 kB 16.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from music21 import *

SyntaxError: invalid syntax (1031709398.py, line 2)

In [30]:
import pandas as pd

In [3]:
s = corpus.parse('bach/bwv65.2.xml')

In [4]:
s.analyze('key')

<music21.key.Key of a minor>

In [5]:
from music21 import corpus
mazurka=corpus.parse('chopin/mazurka06-2')

In [6]:
mazurka.analyze('key')

<music21.key.Key of c# minor>

In [18]:
configure.run()


______________________________________________________________________________ 
  
Welcome to the music21 Configuration Assistant. You will be guided through a 
number of questions to install and set up music21. Simply pressing return at a 
prompt will select a default, if available. 

You may run this configuration again at a later time by running 
music21/configure.py. 
  
______________________________________________________________________________ 
  
Defining an XML Reader permits automatically opening music21-generated 
MusicXML in an editor for display and manipulation when calling the show() 
method. Setting this option is highly recommended. 
  
[1] /Applications/MuseScore 3.app 
  
Choose a number from the preceding options (default is 1): 
MusicXML Reader set to: /Applications/MuseScore 3.app 
  
______________________________________________________________________________ 
  
The BSD-licensed music21 software is distributed with a corpus of encoded 
compositions which ar

In [13]:
nocturne=converter.parse('/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/test/chopnoc9.mxl', format='musicmxl')

In [14]:
elements=nocturne.flat.notes

number_of_notes = 0
for elem in elements:
    if isinstance(elem, note.Note):
        number_of_notes += 1
    elif isinstance(elem, chord.Chord):
        number_of_notes += len(elem.pitches)



In [15]:
print(f'Total number of notes in the piece: {number_of_notes}')

Total number of notes in the piece: 1253


In [ ]:
num_measures = len(nocturne.recurse().getElementsByClass('Measure'))
print(num_measures/2)
#need to divide measures by 2 because it counts the left and right hand parts separately.

38.0


In [57]:
from collections import Counter

duration_counter = Counter()

for note in nocturne.recurse().notes:
    duration_counter[note.duration.quarterLength] += 1

num_32nd_notes_and_shorter = sum(count for duration, count in duration_counter.items() if duration <= 0.125)
print(num_32nd_notes_and_shorter)

num_16_notes=sum(count for duration, count in duration_counter.items() if duration==.25)
print(num_16_notes)

num_eighth=sum(count for duration, count in duration_counter.items() if duration==0.5)
print(num_eighth)

num_quarter=sum(count for duration, count in duration_counter.items() if duration==1)
print(num_quarter)

num_half=sum(count for duration, count in duration_counter.items() if duration==2)
print(num_half)

sample= [106, 122, 494, 20, 0]

106
122
494
20
0


In [70]:
# Define note durations
note_durations = {
    '32nd_and_shorter': 0.125,
    '16th': 0.25,
    'eighth': 0.5,
    'quarter': 1,
    'dotted quarter': 1.5,
    'half': 2,
    'dotted half': 3,
    'whole': 4
}

duration_counter = Counter(note.duration.quarterLength for note in nocturne.recurse().notes)

def count_notes(nocturne, duration_counter):
    note_counts= {}
    for name, duration in note_durations.items():
        if name=='32nd_and_shorter':
            note_counts[name]= sum(count for d, count in duration_counter.items() if d<=duration)
        else:
            note_counts[name]=duration_counter[duration]
    return note_counts

note_counts = count_notes(nocturne, duration_counter)

for name, count in note_counts.items():
    print(f"{name}: {count}")

total_notes = sum(note_counts.values())
print(total_notes)

32nd_and_shorter: 106
16th: 122
eighth: 494
quarter: 20
dotted quarter: 48
half: 0
dotted half: 9
whole: 0
799
